## Compute Graph Features on a Biomedical Graph (Hetionet)

In this example notebooks we will use Hetionet (https://het.io/) as an example of biomedical knowledge graph to illustrate how to derive descriptive graph features to be used in skywalkR or other recommendation systems.

Hetionet is an integrative network of biomedical knowledge assembled from 29 different databases of genes, compounds, diseases, and more. The network combines over 50 years of biomedical information into a single resource, consisting of 47,031 nodes (11 types) and 2,250,197 relationships (24 types).

Hetionet edges can be obtained there: https://github.com/hetio/hetionet/tree/master/hetnet/tsv

Unzip the edge TSV and change the following hetionet_path to the location of your edge file.

In [55]:
hetionet_path = "/projects/it/proj/bikg/HETIONET/hetionet-v1.0-edges.sif"

First we load up the edges in a pandas dataframe:

In [56]:
import pandas as pd

hetionet_edges_df = pd.read_csv(hetionet_path, sep="\t")

We can check that we have all of the 2,250,197 edges:

In [57]:
hetionet_edges_df.shape

(2250197, 3)

And sample the content:

In [58]:
hetionet_edges_df.sample(5)

,source,metaedge,target
987607,Gene::80731,GcG,Gene::80255
795802,Anatomy::UBERON:0002240,AdG,Gene::7051
835400,Anatomy::UBERON:0000007,AdG,Gene::3212
1327952,Gene::55082,Gr>G,Gene::23406
1456930,Gene::9123,Gr>G,Gene::23399


### Building the graph on GPU

We will now use cugraph (part of the RAPIDS software suite) to assemble these edge in a graph on GPU. It might seem unecessary for a small graph like Hetionet but makes a big difference for larger graphs.

In [59]:
import cugraph as cg

G = cg.MultiDiGraph()
G.from_pandas_edgelist(hetionet_edges_df, source="source", destination="target")

We can verify that the GPU VRAM is used (you should see approximately 1.2GB VRAM usage):

In [60]:
!nvidia-smi

Fri Nov 12 21:04:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:04:00.0 Off |                    0 |
| N/A   32C    P0    34W / 250W |   1489MiB / 16160MiB |     32%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:05:00.0 Off |                    0 |
| N/A   

And check the number of edges and nodes:

In [61]:
print("Number of edges: ", len(G.edges()))
print("Number of nodes: ", len(G.nodes()))

Number of edges:  2250197
Number of nodes:  45158


## Extract descriptive graph-based features:

Here we demonstrate how to compute the following graph features: 

- **pagerank** ~ a measure of 'popularity' of a node; PageRank is an algorithm used to rank websites in Google’s search results. It counts the number, and quality, of links to a page which determines an estimation of how important the page is. The underlying assumption is that pages of importance are more likely to receive a higher volume of links from other pages.

- **betweenness** ~ betweenness centrality is a way of detecting the amount of influence a node has over the flow of information in a graph. It is often used to find nodes that serve as a bridge from one part of a graph to another;

- **number of edges connected to a node** ~ one of basic structural properties;

- **number of unique neighbours** ~ another basic strutural property.

- **distance in embedding space** ~ given a knowledge graph we can learn graph representation for each node, so that a node is mapped to a vector of numbers (embedding). We can then use embeddings to conviently compute distances (similarities) between nodes in the graph.

### PageRank

In [62]:
%%time
pr_df = cg.pagerank(G)
pr_df.sample(5)

CPU times: user 15.1 ms, sys: 1.25 ms, total: 16.3 ms
Wall time: 15 ms


,pagerank,vertex
39806,0.000023,Molecular Function::GO:0015923
7121,0.000014,Gene::152078
596,0.000016,Gene::4935
25250,0.000021,Gene::26271
34152,0.000014,Side Effect::C0854146


### Betweenness Centrality

In [63]:
%%time
bc_df = cg.betweenness_centrality(G)
bc_df.sample(5)

CPU times: user 1min 23s, sys: 25.6 s, total: 1min 49s
Wall time: 1min 48s


,betweenness_centrality,vertex
30807,0.000000,Side Effect::C1096116
33517,0.000000,Gene::727851
18613,0.000000,Pharmacologic Class::N0000008016
1525,0.000000,Gene::400629
9403,0.000009,Gene::89849


### Number of edges connected to a node

First we compute the in- and out- degrees of each nodes

In [64]:
%%time
degs_df = G.degrees()
degs_df.sample(5)

CPU times: user 8.19 ms, sys: 1.56 ms, total: 9.76 ms
Wall time: 8.3 ms


,in_degree,out_degree,vertex
42676,33,36,Gene::27130
2284,3,0,Side Effect::C0262593
10569,35,44,Gene::3177
684,1,0,Side Effect::C0235564
19070,25,0,Biological Process::GO:0045577


In [65]:
degs_df["n_edges"] = degs_df["in_degree"]+degs_df["out_degree"]
degs_df.sample(5)

,in_degree,out_degree,vertex,n_edges
36069,100,0,Biological Process::GO:0044724,100
41113,64,32,Gene::153769,96
29358,3,0,Biological Process::GO:0097091,3
40353,103,0,Biological Process::GO:0061387,103
35563,17,13,Gene::340547,30


### Number of unique neighbours

For this feature we will reuse the degree method, but this time we will force the graph to be undirected and without multiedges, effectively counting the number of neighbours.

In [66]:
simpleG = cg.Graph(G)

In [67]:
uniq_neigh_df = simpleG.degrees()
uniq_neigh_df.sample(5)

,in_degree,out_degree,vertex
38180,14,14,Biological Process::GO:0035864
835,339,339,Gene::55294
40763,128,128,Gene::117144
21063,8,8,Biological Process::GO:0071501
20092,74,74,Compound::DB00720


Since we have an undirected graph, both in and out degrees can be used as a count of neighbours

### Embedding distance

For this example we will use a simple node-embedding technique provided via the karateclub package, SocioDim. For more informations on this package, please visit https://karateclub.readthedocs.io/en/latest/index.html

In [68]:
import networkx as nx
from karateclub.node_embedding.neighbourhood import SocioDim


cugraph is not a backend supported by karateclub so we will have to resort to networkx for this part of the example:

In [69]:
hetionet_nx = nx.Graph(hetionet_edges_df)

And we must reindex the nodes for karateclub:

In [95]:
reindexed_hetionet = nx.relabel.convert_node_labels_to_integers(hetionet_nx, first_label=0, ordering='default', label_attribute='old_label')

And keep a mapping to the labels:

In [99]:
node_mapping = nx.get_node_attributes(reindexed_hetionet, "old_label")

For example, let's assume node 100 is our gene of interest:

In [100]:
node_mapping[100]

'Gene::10673'

Now we train a 2-dimensionnal node embedding of Hetionet:

In [89]:
model = SocioDim(dimensions=2)
model.fit(reindexed_hetionet)
embedding = model.get_embedding()

In [90]:
embedding.shape

(45158, 2)

As an example, we can retrieve the embedding of the gene found in position 100:

In [102]:
gene_emb = embedding[0]
gene_emb

array([0.00097192, 0.0005156 ])

### Computing distance to the gene of interest in the embedding space

In [19]:
import numpy as np

In [104]:
L2_distance_to_gene = np.linalg.norm(np.subtract(embedding, disease), axis=1)

In [105]:
L2_distance_to_gene

array([4.74048727e-03, 5.46651015e-03, 7.85429459e-18, ...,
       4.44516781e-03, 4.22786202e-03, 4.56106692e-03])